In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import import_ipynb
import copy
import data_preperation as dp
import internal_inconsistencies as ii
import external_inconsistencies as ei


importing Jupyter notebook from data_preperation.ipynb


In [ ]:
countries={ 'Austria': 'AT', 'Belgium': 'BE',  'Bulgaria': 'BG', 'Switzerland': 'CH', 'Czech Republic': 'CZ', 'Germany': 'DE', 'Denmark': 'DK', 'Estonia': 'EE', 'Spain': 'ES', 'Finland': 'FI', 'France': 'FR',  'Greece': 'GR', 'Hungary': 'HU', 'Ireland': 'IE', 'Italy': 'IT', 'Lithuania': 'LT', 'Latvia': 'LV', 'Montenegro': 'ME','Netherlands': 'NL', 'Norway': 'NO', 'Poland': 'PL', 'Portugal': 'PT', 'Serbia': 'RS', 'Sweden': 'SE', 'Slovenia': 'SI', 'Slovakia': 'SK', 'United Kingdom': 'GB'}
abbr_list=list(countries.values())

# 1. Data mismatches in ENTSO-E data

## 1.1 Calculating missing values in data files

In [ ]:
generation_load_missing_data, transmission_missing_data, mismatch_data = ii.mismatch_raw_data(countries)

# 2. ENTSO-E data representation

## 2.1 Get all the hourly load data of each country to a single dataframe

In [ ]:
load_dic=dp.load(countries)
# display(load_data)

## 2.2 Get all the hourly generation data of each country from each source to a single dataframe

In [ ]:
generation_dic=dp.generation(countries)
# display(generation_data)

## 2.3 Get all the hourly cross-border transmission data of each country to a single dataframe

In [ ]:
transmission_data = dp.cross_border()
# display(transmission_data)
# 'cross_border_data' shows the net import/net export of the whole country in hourly timesteps
# display(crossborder_data)


# 3. Consolidation based on internal data

## 3.1 Obtaining consolidated ENTSO-E data using Nearest Neighbours Mean Approach

### !!! This step takes about 14 minutes to complete in an Intel I5 processor with 8 GB Ram

In [ ]:
ii.consolidation(countries, 'Nearest Neighbours Mean')


## 3.2 Obtaining consolidated data using Polynomial Linear Regression

### !!! This step takes about 40 minutes to complete in an Intel I5 processor with 8 GB Ram

In [ ]:
ii.consolidation(countries, 'Polynomial Linear Regression')


## 3.3 Obtaining consolidated ENTSO-E data using Internal Sigma Approach

### !!! This step takes about 2 hours to complete in an Intel I5 processor with 8 GB Ram

In [3]:
load_dic = {}
generation_dic = {}
transmission_data = pd.DataFrame()

for abbr in abbr_list:
    load_dic[abbr] = pd.read_csv(f'../Data Sources/output/Polynomial Linear Regression/Load/{abbr}.csv').iloc[:,1:]
    generation_dic[abbr] = pd.read_csv(f'../Data Sources/output/Polynomial Linear Regression/Generation/{abbr}.csv').iloc[:,1:]
transmission_data = pd.read_csv(f'../Data Sources/output//Polynomial Linear Regression/Transmission/all_transmissions.csv').iloc[:, 1:]

In [4]:
load_dic_copy = copy.deepcopy(load_dic) 
generation_dic_copy = copy.deepcopy(generation_dic)
transmission_data_copy = copy.deepcopy(transmission_data)
consolidated_gen_data, consolidated_load_data, consolidated_transmission_data = ii.sigma(load_dic_copy, generation_dic_copy, transmission_data_copy, countries)


SIGMA CALCULATED
VARIABLES DECLARED
Academic license - for non-commercial use only - expires 2021-09-28
Using license file C:\Users\lovin\gurobi.lic
OPTIMIZATION COMPLETED


<string>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## 3.2 Mismatch analysis in the consolidated ENTSO-E data

### 3.2.1 Mismatch analysis in all 27 countries

In [ ]:
ii.mismatch_analysis(mismatch_data, abbr_list)


### 3.2.2 Mismatch analysis with only selected countries in optimization based internal consolidation method

In [ ]:
selected_countries = ['AT', 'CZ', 'DK', 'ES', 'FR','IE', 'NL', 'NO', 'PT', 'SI', 'SE', 'CH', 'UK']
ii.mismatch_analysis(mismatch_data, selected_countries)


# 4. Comparission of ENTSO-E data with other sources

## 4.1 Comparing ENTSO-E & Eurostat data

### 4.1.1 Comparing annual generation data of ENTSO-E & Eurostat data

In [ ]:
# ei.eurostat_comparission_gen_data(generation_data,countries)

### 4.1.2 Comparing annual load data of ENTSO-E & Eurostat data

In [ ]:
# ei.eurostat_comparission_load_data(load_data, countries)


### 4.1.3 Comparing annual import/export data of ENTSO-E & Eurostat data

In [ ]:
# ei.eurostat_comparission_transmission_data(import_export_using_crossborder_data, countries)